In [1]:
"""
File : data_merge.ipynb
Created by : Amen Ouannes
Brief : Create the dataset for the training

"""

'\nFile : data_merge.ipynb\nCreated by : Amen Ouannes\nBrief : Create the dataset for the training\n\n'

In [1]:
#import libraries
import data_O as tool
import os as os
import pandas as pd
import pickle as pk
from datetime import datetime, timedelta
print(os.getcwd())

/data/amen/amen_env/data_processing


In [2]:
#associate macs to calf numbers
macs= {"D1:B6:FC:34:99:3A" : 7480, "F4:CC:E1:C5:8A:A9" :8855,
       "FB:E5:24:5B:8A:68" : 8851, "D0:C9:50:5E:88:F8" :8846, "D3:6B:03:69:94:4E" : 8854}

In [3]:
#extract data from pickles 
with open("DAL.pkl", 'rb') as f:
    DAL = pk.load(f)
with open("dataset.pkl", 'rb') as f:
    calf_1 = pk.load(f)
    calf_4 = pk.load(f)
    calf_11 = pk.load(f)
    calf_7 = pk.load(f)
    calf_3 = pk.load(f)
with open('AX3.pkl', 'rb') as f:
    AX3 = pk.load(f)

In [4]:
#concat all the calves in one dataframe
ruuvi = pd.concat([calf_1, calf_4, calf_11, calf_7, calf_3], ignore_index=True)
assert (len(ruuvi) == len(calf_1) + len(calf_4) + len(calf_11) + len(calf_7) + len(calf_3))
print("data concatenated successfully")

data concatenated successfully


In [5]:
# add a column that contains the calfnumber based on the mac addresses
ruuvi['calfNumber'] = ruuvi['mac'].map(macs)
print(ruuvi['calfNumber'].value_counts())

calfNumber
7480    585956
8855    584282
8851    584209
8846    583621
8854    577736
Name: count, dtype: int64


In [6]:
#Merge ruuvi and DAL
copy_ruuvi = ruuvi.copy()
drink = pd.merge(copy_ruuvi, DAL, on=['time', 'calfNumber'])
drink['state'] = 'drink'
print(drink)
#merge ruuvi and AX3
copy_ruuvi = ruuvi.copy()
up_down = pd.merge(copy_ruuvi, AX3, on = ['time', 'calfNumber'])
print(up_down)
up_down['state'].replace({1: 'up', 0: 'down'}, inplace=True)
up_down['state'].value_counts()

                      time     acc_x     acc_y     acc_z  st_dev_x  st_dev_y  \
0      2024-05-04 05:42:29 -0.996800 -0.144000  0.110400  0.243009  0.242065   
1      2024-05-04 05:42:30 -1.034182 -0.119273  0.024727  0.265163  0.118522   
2      2024-05-04 05:42:31 -0.966000 -0.312000  0.078000  0.142580  0.107161   
3      2024-05-04 05:42:32 -0.969333 -0.418667 -0.002667  0.117864  0.054080   
4      2024-05-04 05:42:33 -0.988444 -0.416000 -0.010667  0.144173  0.050596   
...                    ...       ...       ...       ...       ...       ...   
180476 2024-05-10 20:04:08 -0.931200  0.192000  0.131200  0.159964  0.141710   
180477 2024-05-10 20:04:09 -0.926545  0.372364  0.062545  0.097968  0.277321   
180478 2024-05-10 20:04:10 -0.871111  0.412444 -0.017778  0.273772  0.195596   
180479 2024-05-10 20:04:11 -0.830222  0.570667 -0.174222  0.133493  0.126491   
180480 2024-05-10 20:04:12 -0.875200  0.492800 -0.134400  0.140915  0.103637   

        st_dev_z     var_x     var_y   

/tmp/ipykernel_2653266/2698269466.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  up_down['state'].replace({1: 'up', 0: 'down'}, inplace=True)


state
down    2055996
up       859808
Name: count, dtype: int64

In [7]:
#exist function to call with the drinking status
def exist(data, time, calf):
    line = data[(data['time'] == time )& (data['calfNumber'] == calf)]
    if not line.empty:
        return True
    return False
print(exist(AX3, ruuvi.iloc[1]['time'], ruuvi.iloc[1]['calfNumber']))
#ensure data balance:
def sample_group(group, num_samples):
    return group.sample(n=num_samples, random_state = 581)

True


In [8]:
#eliminate rows that verify drink within stand
drink_copy = drink.copy()
#up = up_down[up_down['state'] == 'up']
common_rows = pd.merge(drink_copy, up_down, on=['time', 'calfNumber'], how='inner')
up_down = up_down[~up_down[['time', 'calfNumber']].isin(common_rows[['time', 'calfNumber']].to_dict(orient='list')).all(axis=1)]
print(up_down)
#filter the down rows together 
#down = up_down[up_down['state'] == 'down']

                       time     acc_x     acc_y     acc_z  st_dev_x  st_dev_y  \
0       2024-05-04 00:00:00 -0.444000 -0.040000 -0.932000  0.015319  0.009238   
1       2024-05-04 00:00:01 -0.436800 -0.036800 -0.924800  0.007729  0.007729   
2       2024-05-04 00:00:02 -0.446000 -0.030000 -0.926000  0.005657  0.010254   
3       2024-05-04 00:00:03 -0.432000 -0.036923 -0.916923  0.006532  0.007686   
4       2024-05-04 00:00:04 -0.440000 -0.035200 -0.929600  0.011314  0.010119   
...                     ...       ...       ...       ...       ...       ...   
2915799 2024-05-10 23:59:56 -0.678400  0.350400  0.601600  0.015457  0.019156   
2915800 2024-05-10 23:59:57 -0.679111  0.339556  0.606222  0.014111  0.017487   
2915801 2024-05-10 23:59:58 -0.699200  0.296000  0.569600  0.109833  0.161284   
2915802 2024-05-10 23:59:59 -0.667200  0.328000  0.617600  0.020027  0.017282   
2915803 2024-05-11 00:00:00 -0.659200  0.345600  0.624000  0.013387  0.014311   

         st_dev_z     var_x

In [9]:
#ensure data balance within drink
print(drink['calfNumber'].value_counts())
length = len(drink[drink['calfNumber'] == 8846])
drink_balanced =drink.groupby('calfNumber').apply(sample_group, num_samples=length).reset_index(drop=True)
print(drink_balanced['calfNumber'].value_counts())

calfNumber
7480    96705
8851    54179
8855    12814
8854     9986
8846     6797
Name: count, dtype: int64
calfNumber
7480    6797
8846    6797
8851    6797
8854    6797
8855    6797
Name: count, dtype: int64


/tmp/ipykernel_2653266/1245723858.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  drink_balanced =drink.groupby('calfNumber').apply(sample_group, num_samples=length).reset_index(drop=True)


In [10]:
#define variables to randomize & sample
num_samples = len(drink_balanced)
num_samples_per_group = num_samples // 5

# Perform stratified sampling for up
up = up_down[up_down['state'] == 'up']
sampled_up = up.groupby('calfNumber').apply(sample_group, num_samples=num_samples_per_group).reset_index(drop=True)
print(sampled_up['calfNumber'].value_counts())
assert (len(drink_balanced) == len(sampled_up[sampled_up['calfNumber'] == 8851])*5)

# Perform stratified sampling for down
down = up_down[up_down['state'] == 'down']
sampled_down = down.groupby('calfNumber').apply(sample_group, num_samples=num_samples_per_group).reset_index(drop=True)
print(sampled_down['calfNumber'].value_counts())
assert (len(drink_balanced) == len(sampled_down[sampled_down['calfNumber'] == 7480])*5)

/tmp/ipykernel_2653266/3748755512.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_up = up.groupby('calfNumber').apply(sample_group, num_samples=num_samples_per_group).reset_index(drop=True)


calfNumber
7480    6797
8846    6797
8851    6797
8854    6797
8855    6797
Name: count, dtype: int64
calfNumber
7480    6797
8846    6797
8851    6797
8854    6797
8855    6797
Name: count, dtype: int64


/tmp/ipykernel_2653266/3748755512.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_down = down.groupby('calfNumber').apply(sample_group, num_samples=num_samples_per_group).reset_index(drop=True)


In [11]:
#Now that we are sure we did avoid any overlaps and balanced our data 
#let's concatenate all

dataset = pd.concat([drink_balanced, sampled_up, sampled_down], ignore_index=True)
print(dataset)
dataset['state'].value_counts()
crosstab_df = pd.crosstab(dataset['state'], dataset['calfNumber'])
print(crosstab_df)

                      time     acc_x     acc_y     acc_z  st_dev_x  st_dev_y  \
0      2024-05-10 07:00:13 -0.934400 -0.059200  0.465600  0.063643  0.039947   
1      2024-05-10 07:10:09 -0.836800  0.017600  0.572800  0.078038  0.034932   
2      2024-05-09 12:58:39 -0.858667  0.176000  0.485333  0.081107  0.046772   
3      2024-05-09 18:59:39 -0.883200  0.190400  0.462400  0.034399  0.020587   
4      2024-05-10 11:54:14 -0.953600  0.254400  0.286400  0.056543  0.090651   
...                    ...       ...       ...       ...       ...       ...   
101950 2024-05-10 00:09:23 -0.629818 -0.525091  0.615273  0.010787  0.009648   
101951 2024-05-05 13:29:11 -0.545600 -0.307200  0.792000  0.015911  0.010119   
101952 2024-05-06 22:41:24 -0.721455 -0.477091  0.554182  0.070274  0.034180   
101953 2024-05-08 19:54:05 -0.615273 -0.823273  0.229818  0.016572  0.016572   
101954 2024-05-07 05:06:38 -0.668444 -0.536889  0.570667  0.017487  0.011624   

        st_dev_z     var_x     var_y   

In [14]:
from datetime import datetime 
def trunc(data, dt1, dt2):
    dt1 = datetime.strptime(dt1, "%Y-%m-%d %H:%M:%S.%f")
    dt2 = datetime.strptime(dt2, "%Y-%m-%d %H:%M:%S.%f")
    sample = data[(data['time'] >= dt1)
                            & (data['time'] <= dt2)]
    return sample
calf1 = dataset[dataset['calfNumber'] == 8854]
print(trunc(calf1, "2024-05-04 02:00:00.0", "2024-05-04 03:00:00.0")['state'])

88415    down
88535    down
88564    down
88591    down
88596    down
         ... 
94436    down
94539    down
94562    down
94701    down
95104    down
Name: state, Length: 66, dtype: object


In [ ]:
#keep only interesting data for the train
dataset.drop(['mac', 'calfNumber', 'time'], axis = 1, inplace = True)

In [ ]:
print(dataset)

In [ ]:
#Mix randomly to ensure the functionnality of train
dataset = dataset.sample(frac= 1 , random_state=581).reset_index(drop=True)
dataset.dropna(inplace = True)
print(dataset)

In [ ]:
#save to pickle 
tool.download(dataset, "train_datax.pkl")
dataset.to_csv("train_data.csv")
print("data downloaded!")